In [1]:
import pandas as pd
import numpy as np

import regex as re
import pickle

from datetime import datetime
from time import sleep

from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
# from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

# from pybliometrics.scopus import AuthorSearch


In [2]:
# Restarted the PC, re-loading variables:

with open("new_author_out_list", "rb") as fp:
    new_author_out_list = pickle.load(fp)

with open("new_first_author_df", "rb") as fp:
    new_first_author_df = pickle.load(fp)

# with open("new_author_out_list_loop_save", "rb") as fp:
#     new_author_out_list_loop_save = pickle.load(fp)

In [2]:
# Restarted the PC, re-loading variables:

# with open("k5k_author_out_list", "rb") as fp:
#     k5k_author_out_list = pickle.load(fp)

with open("k5k_first_author_df", "rb") as fp:
    k5k_first_author_df = pickle.load(fp)

# with open("new_author_out_list_loop_save", "rb") as fp:
#     new_author_out_list_loop_save = pickle.load(fp)

In [3]:
scopus_codes_dict = {'Agricultural and Biological Sciences': 'AGRI',
 'Arts and Humanities': 'ARTS',
 'Biochemistry, Genetics and Molecular Biology': 'BIOC',
 'Business, Management and Accounting': 'BUSI',
 'Chemical Engineering': 'CENG',
 'Chemistry': 'CHEM',
 'Computer Science': 'COMP',
 'Decision Sciences': 'DECI',
 'Dentistry': 'DENT',
 'Earth and Planetary Sciences': 'EART',
 'Economics, Econometrics and Finance': 'ECON',
 'Energy': 'ENER',
 'Engineering': 'ENGI',
 'Environmental Science': 'ENVI',
 'Health Professions': 'HEAL',
 'Immunology and Microbiology': 'IMMU',
 'Materials Science': 'MATE',
 'Mathematics': 'MATH',
 'Medicine': 'MEDI',
 'Neuroscience': 'NEUR',
 'Nursing': 'NURS',
 'Pharmacology, Toxicology and Pharmaceutics': 'PHAR',
 'Physics and Astronomy': 'PHYS',
 'Psychology': 'PSYC',
 'Social Sciences': 'SOCI',
 'Veterinary': 'VETE',
 'Multidisciplinary': 'MULT'}

def sbj_area_query_creator(sbj_list):
    query = " AND ".join([f"SUBJAREA({scopus_codes_dict[sbj]})" for sbj in sbj_list])
    return query

To speed up the Scopus API step, will try to start and create a second loop in this nb.

Will use new_additions_df as the source article dataset to avoid duplications.

### Part 1: author_df & first_author_df

In [3]:
with open("7e_variables/new_additions_df", "rb") as fp:
    new_additions_df = pickle.load(fp)

In [9]:
new_author_df = new_additions_df.dropna(subset= "author").reset_index().loc[:,["DOI", "JRNL_ID", "author"]].explode("author").reset_index(drop=True)

new_aut_df = pd.DataFrame(list(new_author_df['author']))

In [11]:
# Combining the two datasets:
new_author_df = pd.concat([new_author_df, new_aut_df], axis=1)

new_author_df.drop(["suffix", "name", "author"], axis=1, inplace=True)

In [21]:
# Drop empty given & family:
new_author_df.dropna(subset=["family", "given"], inplace=True)

In [22]:
# Need to df[0] = df[0].str.strip() first:
new_author_df.given = new_author_df.given.str.strip()
new_author_df.family = new_author_df.family.str.strip()

# Remove multiple spaces:
new_author_df.given = new_author_df.given.map(lambda x: re.sub(' +', ' ', x))
new_author_df.family = new_author_df.family.map(lambda x: re.sub(' +', ' ', x))


# Case 1: A. C
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x)))].given.map(lambda x: x +".")
# Case 2: A.C
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x)))].given.map(lambda x: x.replace(".",". ") +".")
# Case 3: AC
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x)))].given.map(lambda x: x[0] + ". " + x[1] + ".")
# Case 4: A C
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x)))].given.map(lambda x: x.replace(" ", ". ") + ".")
# Case 5: A.C.
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x))), "new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x)))].given.map(lambda x: x.replace(".", ". ").strip())

# Single letter names:
new_author_df.loc[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x))),"new_given"] = new_author_df[new_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x)))].given.map(lambda x: x + ".")



In [51]:
# Removing [Name], (Name),
# remove "Name" , “Name”,
# remove & % , - ' .
# simplf. ö & ü & � & i

# Main Preprocessing:
def name_simplifier(pdf_read):
    # Lowercase all:
    pdf_modified = pdf_read.lower()
    # Remove everything in brackets & paranthesis:
    pdf_modified = re.sub("[\(\[\{].*?[\)\]\}]", "", pdf_modified)
    # Remove quote names:
    pdf_modified = re.sub('["“].*?["”]', "", pdf_modified)
    # Remove numbers
    pdf_modified = re.sub(r'[0-9]', '', pdf_modified)
    # Remove weird punct.
    pdf_modified = re.sub(r'[&\?\$\+\\\*\^\|]', '', pdf_modified)
    # Simplify acct. a:
    pdf_modified = re.sub(r'[áạàảãăặằẳẵâấậầẩẫā]', 'a', pdf_modified)
    # Simplify acct. i:
    pdf_modified = re.sub(r'[íịìỉĩïǐĭīĩįɨıî]', 'i', pdf_modified)
    # Simplify acct. i:
    pdf_modified = re.sub(r'[éẹèẻẽêếệềểễ]', 'e', pdf_modified)
    # Simplify acct. o:
    pdf_modified = re.sub(r'[óòȯôöǒŏōõǫőốồøṓṑ]', 'o', pdf_modified)
    # Simplify acct. u:
    pdf_modified = re.sub(r'[úùûüǔŭūũů]', 'u', pdf_modified)
    # Remove multiple spaces:
    pdf_modified = re.sub(' +', ' ', pdf_modified)
    # Return pdf_modified
    return pdf_modified



In [24]:
# Simpli:

new_author_df.loc[:,"new_given"] = new_author_df.given.map(name_simplifier)
new_author_df.loc[:,"new_family"] = new_author_df.family.map(name_simplifier)

In [25]:
# After process stripping:

new_author_df.given = new_author_df.given.str.strip()
new_author_df.family = new_author_df.family.str.strip()

new_author_df.new_given = new_author_df.new_given.str.strip()
new_author_df.new_family = new_author_df.new_family.str.strip()



In [26]:
with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

new_author_df = pd.merge(left=new_author_df, right=q1_jrnl_df["SA_list"], how="left", left_on="JRNL_ID", right_index=True)

In [28]:
# Creating a first author df
# Using .groupby() to have DOI & SA_list columns 


new_first_author_df = new_author_df[new_author_df.sequence == "first"].groupby(["new_given", "new_family"])["SA_list"].apply(list)
new_first_author_df = new_first_author_df.to_frame().merge(new_author_df[new_author_df.sequence == "first"].groupby(["new_given", "new_family"])["DOI"].apply(list), how= "inner", right_index=True, left_index=True)
new_first_author_df.reset_index(inplace=True)

In [40]:
new_first_author_df[new_first_author_df.new_given.map(lambda x: bool(re.match("\W",x)))]

,new_given,new_family,SA_list,DOI
24266,‘fauxparl’,,"[[Health Professions, Social Sciences]]",[10.1080/09687599.2020.1838262]


In [41]:
new_first_author_df[new_first_author_df.new_family.map(lambda x: bool(re.match("\W",x)))]

,new_given,new_family,SA_list,DOI


In [42]:
# Problematic DOI: 10.1080/09687599.2020.1838262

new_first_author_df.drop(24266, inplace=True)

In [44]:
# 1- Flattening SA_list col:
def flatten(l):
    return [item for sublist in l for item in sublist]

new_first_author_df.loc[:,"SA_list"] = new_first_author_df.SA_list.map(flatten)

# 2- Creating an Author ID col:
new_first_author_df.reset_index(drop=True, inplace=True)
new_first_author_df.reset_index(drop=False,inplace=True)
new_first_author_df.loc[:,"index"] = new_first_author_df.index.map(lambda x: "aut_" + str(x))
new_first_author_df.set_index("index", inplace=True)

In [45]:
with open("new_first_author_df","wb") as p:
    pickle.dump(new_first_author_df, p)

with open("new_author_df","wb") as p:
    pickle.dump(new_author_df, p)

READY!

### Part 2: Scopus API Loop

In [56]:
new_author_input_dict = new_first_author_df[["new_given","new_family","SA_list"]].to_dict("index")

In [58]:
# Full Loop - First:
# Fixed API keys lists

new_author_out_list= list()

avail_api_keys_list = ["b7e95f5ea731eb5c9e84e7a1d499f50e","8707db153e4b9672fa6df25b03a5f747",
"2dab4694b4347fa574d159bb97484fc4","6af3d2c09eb08ec6e12f0cead9a1f5bb","3872e798bf48fa28af583b9ebef5deb6","1473c31dbcdb425f9cdaf75c673279d3"]

used_api_keys_list = list()

client = ElsClient("30195f0b2192052a36bcab9ce3c4064f")
print(f'Start Time = {datetime.now().strftime("%H:%M:%S")}')

for auth_id in new_author_input_dict:
    given_in = new_author_input_dict[auth_id]["new_given"]
    family_in = new_author_input_dict[auth_id]["new_family"]
    sa_in = new_author_input_dict[auth_id]["SA_list"]

    try:
        auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
        auth_srch.execute(client)
    except:
        if len(avail_api_keys_list) >0:
            with open("new_author_out_list_loop_save","wb") as p:
                pickle.dump(new_author_out_list, p)
            print("Loop save complete!")
            new_key = avail_api_keys_list.pop(0)            
            client = ElsClient(new_key)
            
            print(f'Time = {datetime.now().strftime("%H:%M:%S")}')
            
            print(f"Changed API key, now using key: {new_key}")
            used_api_keys_list.append(new_key)

            auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
            auth_srch.execute(client)

        else:
            print("No available API keys to use!")
            break


    country_out = list()
    given_out = list()
    family_out = list()
    doc_count_out = list()

    for auth in auth_srch.results:
        # Country:
        try:
            country_out.append(auth['affiliation-current']['affiliation-country'])
        except:
            pass

        # Given:
        try:
            given_out.append(auth['preferred-name']['given-name'])
        except:
            pass

        # Family:
        try:
            family_out.append(auth['preferred-name']['surname'])
        except:
            pass

        # Doc Count:
        try:
            doc_count_out.append(auth['document-count'])
        except:
            pass

    
    author_out_dict = dict(author_id=auth_id, country=country_out, given=given_out, family=family_out, doc_count=doc_count_out)

    new_author_out_list.append(author_out_dict)




Start Time = 12:58:34
Loop save complete!
Time = 16:26:45
Changed API key, now using key: b7e95f5ea731eb5c9e84e7a1d499f50e
Loop save complete!
Time = 16:41:40
Changed API key, now using key: 8707db153e4b9672fa6df25b03a5f747


HTTPError: HTTP 500 Error from https://api.elsevier.com/content/search/author?query=AUTHLAST%28los%29+AND+AUTHFIRST%28e.+l%E2%80%8A.%29+AND+SUBJAREA%28NEUR%29+AND+SUBJAREA%28BIOC%29+AND+SUBJAREA%28MEDI%29
and using headers {'X-ELS-APIKey': '8707db153e4b9672fa6df25b03a5f747', 'User-Agent': 'elsapy-v0.5.0', 'Accept': 'application/json'}:
{"service-error":{"status":{"statusCode":"GENERAL_SYSTEM_ERROR","statusText":"Error transforming XML with XSL"}}}

In [90]:
with open("new_author_out_list","wb") as p:
    pickle.dump(new_author_out_list, p)

In [89]:
pd.DataFrame(new_author_out_list)

,author_id,country,given,family,doc_count
0,aut_0,"[United States, China, India, Saudi Arabia, Om...","[Md Zakirul Alam, Md Mahbub, Moshahid Alam, Pr...","[Bhuiyan, Alam, Rizvi, Alam, Khan, Sarker, Ala...","[230, 209, 159, 113, 111, 105, 82, 82, 74, 68,..."
1,aut_1,"[Spain, Portugal]","[Antonio, Maria Alcina]","[Alcina, Pereira]","[84, 84]"
2,aut_2,[Jordan],[Abeer],[AlHadidi],[24]
3,aut_3,[United States],"[Andriy, D. A.]","[Andreyev, Andreyev]","[27, 1]"
4,aut_4,[Saudi Arabia],[Abdulkadir Sh],[Aydarous],[27]
...,...,...,...,...,...
17019,aut_17021,[Spain],[Pedro L.],[Valenzuela],[159]
17020,aut_17022,"[Portugal, Portugal]","[Pedro M., Pedro M.]","[Castro, Castro]","[141, 22]"
17021,aut_17023,"[Portugal, Brazil, Portugal]","[Pedro Mendes, Pedro Walfir Martins E., Pedro M.]","[Sousa, Souza-Filho, Sousa]","[104, 92, 31]"
17022,aut_17024,[Portugal],[P. M.P.],[Salomé],[100]


In [85]:
new_author_input_dict = new_first_author_df[["new_given","new_family","SA_list"]][16100:].to_dict("index")

In [86]:
# Full Loop:
# Fixed API keys lists

# new_author_out_list= list()

avail_api_keys_list = ["2dab4694b4347fa574d159bb97484fc4","6af3d2c09eb08ec6e12f0cead9a1f5bb","3872e798bf48fa28af583b9ebef5deb6","1473c31dbcdb425f9cdaf75c673279d3"]

used_api_keys_list = list("8707db153e4b9672fa6df25b03a5f747")

client = ElsClient("b7e95f5ea731eb5c9e84e7a1d499f50e","30195f0b2192052a36bcab9ce3c4064f")

print(f'Start Time = {datetime.now().strftime("%H:%M:%S")}')

for auth_id in new_author_input_dict:
    given_in = new_author_input_dict[auth_id]["new_given"]
    family_in = new_author_input_dict[auth_id]["new_family"]
    sa_in = new_author_input_dict[auth_id]["SA_list"]

    try:
        auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
        auth_srch.execute(client)
    except:
        if len(avail_api_keys_list) >0:
            with open("new_author_out_list_loop_save","wb") as p:
                pickle.dump(new_author_out_list, p)
            print("Loop save complete!")
            new_key = avail_api_keys_list.pop(0)            
            client = ElsClient(new_key)
            
            print(f'Time = {datetime.now().strftime("%H:%M:%S")}')
            
            print(f"Changed API key, now using key: {new_key}")
            used_api_keys_list.append(new_key)

            auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
            auth_srch.execute(client)

        else:
            print("No available API keys to use!")
            break


    country_out = list()
    given_out = list()
    family_out = list()
    doc_count_out = list()

    for auth in auth_srch.results:
        # Country:
        try:
            country_out.append(auth['affiliation-current']['affiliation-country'])
        except:
            pass

        # Given:
        try:
            given_out.append(auth['preferred-name']['given-name'])
        except:
            pass

        # Family:
        try:
            family_out.append(auth['preferred-name']['surname'])
        except:
            pass

        # Doc Count:
        try:
            doc_count_out.append(auth['document-count'])
        except:
            pass

    
    author_out_dict = dict(author_id=auth_id, country=country_out, given=given_out, family=family_out, doc_count=doc_count_out)

    new_author_out_list.append(author_out_dict)




Start Time = 02:56:05
Loop save complete!
Time = 02:56:06
Changed API key, now using key: 2dab4694b4347fa574d159bb97484fc4
Loop save complete!
Time = 03:35:20
Changed API key, now using key: 6af3d2c09eb08ec6e12f0cead9a1f5bb


HTTPError: HTTP 400 Error from https://api.elsevier.com/content/search/author?query=AUTHLAST%28or%29+AND+AUTHFIRST%28peggy%29+AND+SUBJAREA%28ARTS%29
and using headers {'X-ELS-APIKey': '6af3d2c09eb08ec6e12f0cead9a1f5bb', 'User-Agent': 'elsapy-v0.5.0', 'Accept': 'application/json'}:
{"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Error translating query"}}}

### Part 3: Improved Loop

In [47]:
with open("new_author_out_list","wb") as p:
    pickle.dump(new_author_out_list, p)

In [46]:
pd.DataFrame(new_author_out_list)

,author_id,country,given,family,doc_count
0,aut_0,"[United States, China, India, Saudi Arabia, Om...","[Md Zakirul Alam, Md Mahbub, Moshahid Alam, Pr...","[Bhuiyan, Alam, Rizvi, Alam, Khan, Sarker, Ala...","[230, 209, 159, 113, 111, 105, 82, 82, 74, 68,..."
1,aut_1,"[Spain, Portugal]","[Antonio, Maria Alcina]","[Alcina, Pereira]","[84, 84]"
2,aut_2,[Jordan],[Abeer],[AlHadidi],[24]
3,aut_3,[United States],"[Andriy, D. A.]","[Andreyev, Andreyev]","[27, 1]"
4,aut_4,[Saudi Arabia],[Abdulkadir Sh],[Aydarous],[27]
...,...,...,...,...,...
24258,aut_24261,[Poland],[Łukasz],[Markiewicz],[25]
24259,aut_24262,[Poland],[Łukasz],[Rudnicki],[52]
24260,aut_24263,[Czech Republic],[Šárka],[Techlovská],[2]
24261,aut_24264,[Slovenia],[Špela],[Tajnšek],[2]


In [48]:
len(new_first_author_df)

24266

In [43]:
new_first_author_df[["new_given","new_family","SA_list"]][23772:]

,new_given,new_family,SA_list
index,,,
aut_23772,yujiu,zhao,"[Earth and Planetary Sciences, Engineering]"
aut_23773,yujuan,chen,"[Mathematics, Economics, Econometrics and Fina..."
aut_23774,yujuan,li,"[Chemical Engineering, Materials Science]"
aut_23775,yuka,inamochi,[Dentistry]
aut_23776,yuki,adam,"[Social Sciences, Medicine, Psychology]"
...,...,...,...
aut_24261,łukasz,markiewicz,"[Psychology, Arts and Humanities, Decision Sci..."
aut_24262,łukasz,rudnicki,"[Mathematics, Physics and Astronomy]"
aut_24263,šarka,techlovska,"[Neuroscience, Pharmacology, Toxicology and Ph..."


In [44]:
new_author_input_dict = new_first_author_df[["new_given","new_family","SA_list"]][23772:].to_dict("index")

In [45]:
# Full Loop - Improved:

#new_author_out_list= list()

# avail_api_keys_list = ["8707db153e4b9672fa6df25b03a5f747","2dab4694b4347fa574d159bb97484fc4","6af3d2c09eb08ec6e12f0cead9a1f5bb","3872e798bf48fa28af583b9ebef5deb6","1473c31dbcdb425f9cdaf75c673279d3"]

# used_api_keys_list = list()

# current_key = avail_api_keys_list.pop(0)
# client = ElsClient(current_key)

print(f'Start Time = {datetime.now().strftime("%H:%M:%S")}')

for auth_id in new_author_input_dict:
    given_in = new_author_input_dict[auth_id]["new_given"]
    family_in = new_author_input_dict[auth_id]["new_family"]
    sa_in = new_author_input_dict[auth_id]["SA_list"]

    try:
        auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
        auth_srch.execute(client)
        cnt_check = True

    except Exception as e:
        err_no = re.search("\d+",str(e))[0]

        print(f"Encountered Error: {err_no}!")
        print(f'Time = {datetime.now().strftime("%H:%M:%S")}')

        with open("new_author_out_list_loop_save","wb") as p:
                pickle.dump(new_author_out_list, p)
        print("Loop save complete!")

        if "400" in str(e):
            # query error, need to skip
            print(f"Skipping author: {given_in} {family_in} - {auth_id}")
            cnt_check = False

        elif "401" in str(e):
            # VPN error, sleep try again
            sleep(60)
            print("Slept for 60 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                print("Got an error again & breaking the loop ! :(")
                break
            
        elif "429" in str(e):
            # change API key 
            while len(avail_api_keys_list) >0:
                used_api_keys_list.append(current_key)
                current_key = avail_api_keys_list.pop(0)
                client = ElsClient(current_key)
                print(f"Changed API key, now using key: {current_key}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("New key is OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "429":
                        print("New key is also finished!")
                    else:
                        print(f"Skipping author: {given_in} {family_in} - {auth_id}")
                        cnt_check = False
                        break

            if len(avail_api_keys_list) == 0:
                print("No available API keys left & breaking the loop! :(")
                break

        elif "500" in str(e):
            sleep(30)
            print("Slept for 30 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                cnt_check = False
            pass
        
        elif "10060" in str(e):
            cnt_check = False
            for i in range(10):
                sleep(30)
                print("Slept for 30 & trying again!")
                print(f"Trial number: {i+1}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("It is now OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "10060":
                        print(f"It is still {err_no}!")
                    else:
                        print(f"Another error: {err_no}")
            
            if not cnt_check:
                print(f"Couldn't fix {err_no} & breaking the loop! :(")
                break

        
        else:
            print("Unknown error & breaking the loop! :(")
            break



    if cnt_check:
        country_out = list()
        given_out = list()
        family_out = list()
        doc_count_out = list()

        for auth in auth_srch.results:
            # Country:
            try:
                country_out.append(auth['affiliation-current']['affiliation-country'])
            except:
                pass

            # Given:
            try:
                given_out.append(auth['preferred-name']['given-name'])
            except:
                pass

            # Family:
            try:
                family_out.append(auth['preferred-name']['surname'])
            except:
                pass

            # Doc Count:
            try:
                doc_count_out.append(auth['document-count'])
            except:
                pass

        author_out_dict = dict(author_id=auth_id, country=country_out, given=given_out, family=family_out, doc_count=doc_count_out)
        new_author_out_list.append(author_out_dict)




Start Time = 20:41:55


LOOP FINISHED!

In [65]:
with open("new_author_out_list_loop_save","wb") as p:
    pickle.dump(new_author_out_list, p)

### Part 4: k5k datasets

In [49]:
with open("7e_variables/last_5k_run_Sum/k5k_new_succ", "rb") as fp:
    k5k_new_succ = pickle.load(fp)

In [52]:
k5k_author_df = k5k_new_succ.dropna(subset= "author").reset_index().loc[:,["DOI", "JRNL_ID", "author"]].explode("author").reset_index(drop=True)

k5k_aut_df = pd.DataFrame(list(k5k_author_df['author']))

In [53]:
# Combining the two datasets:
k5k_author_df = pd.concat([k5k_author_df, k5k_aut_df], axis=1)

k5k_author_df.drop(["suffix", "name", "author"], axis=1, inplace=True)

In [54]:
# Drop empty given & family:
k5k_author_df.dropna(subset=["family", "given"], inplace=True)

In [55]:
# Need to df[0] = df[0].str.strip() first:
k5k_author_df.given = k5k_author_df.given.str.strip()
k5k_author_df.family = k5k_author_df.family.str.strip()

# Remove multiple spaces:
k5k_author_df.given = k5k_author_df.given.map(lambda x: re.sub(' +', ' ', x))
k5k_author_df.family = k5k_author_df.family.map(lambda x: re.sub(' +', ' ', x))


# Case 1: A. C
k5k_author_df.loc[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x))),"new_given"] = k5k_author_df[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.\s[A-Z]$", x)))].given.map(lambda x: x +".")
# Case 2: A.C
k5k_author_df.loc[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x))),"new_given"] = k5k_author_df[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]$", x)))].given.map(lambda x: x.replace(".",". ") +".")
# Case 3: AC
k5k_author_df.loc[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x))),"new_given"] = k5k_author_df[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]{2}$", x)))].given.map(lambda x: x[0] + ". " + x[1] + ".")
# Case 4: A C
k5k_author_df.loc[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x))),"new_given"] = k5k_author_df[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\s[A-Z]$", x)))].given.map(lambda x: x.replace(" ", ". ") + ".")
# Case 5: A.C.
k5k_author_df.loc[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x))), "new_given"] = k5k_author_df[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]\.[A-Z]\.$", x)))].given.map(lambda x: x.replace(".", ". ").strip())

# Single letter names:
k5k_author_df.loc[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x))),"new_given"] = k5k_author_df[k5k_author_df.given.map(lambda x: bool(re.match("^[A-Z]$", x)))].given.map(lambda x: x + ".")



In [56]:
# Simpli:

k5k_author_df.loc[:,"new_given"] = k5k_author_df.given.map(name_simplifier)
k5k_author_df.loc[:,"new_family"] = k5k_author_df.family.map(name_simplifier)

In [57]:
# After process stripping:

k5k_author_df.given = k5k_author_df.given.str.strip()
k5k_author_df.family = k5k_author_df.family.str.strip()

k5k_author_df.new_given = k5k_author_df.new_given.str.strip()
k5k_author_df.new_family = k5k_author_df.new_family.str.strip()



In [58]:
with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

k5k_author_df = pd.merge(left=k5k_author_df, right=q1_jrnl_df["SA_list"], how="left", left_on="JRNL_ID", right_index=True)

In [59]:
# Creating a first author df
# Using .groupby() to have DOI & SA_list columns 


k5k_first_author_df = k5k_author_df[k5k_author_df.sequence == "first"].groupby(["new_given", "new_family"])["SA_list"].apply(list)
k5k_first_author_df = k5k_first_author_df.to_frame().merge(k5k_author_df[k5k_author_df.sequence == "first"].groupby(["new_given", "new_family"])["DOI"].apply(list), how= "inner", right_index=True, left_index=True)
k5k_first_author_df.reset_index(inplace=True)

In [60]:
k5k_first_author_df[k5k_first_author_df.new_given.map(lambda x: bool(re.match("\W",x)))]

,new_given,new_family,SA_list,DOI


In [61]:
k5k_first_author_df[k5k_first_author_df.new_family.map(lambda x: bool(re.match("\W",x)))]

,new_given,new_family,SA_list,DOI


In [62]:
# 1- Flattening SA_list col:
def flatten(l):
    return [item for sublist in l for item in sublist]

k5k_first_author_df.loc[:,"SA_list"] = k5k_first_author_df.SA_list.map(flatten)

# 2- Creating an Author ID col:
k5k_first_author_df.reset_index(drop=True, inplace=True)
k5k_first_author_df.reset_index(drop=False,inplace=True)
k5k_first_author_df.loc[:,"index"] = k5k_first_author_df.index.map(lambda x: "aut_" + str(x))
k5k_first_author_df.set_index("index", inplace=True)

In [64]:
with open("k5k_first_author_df","wb") as p:
    pickle.dump(k5k_first_author_df, p)

with open("k5k_author_df","wb") as p:
    pickle.dump(k5k_author_df, p)

### Part 5: k5k API Loop

In [4]:
k5k_author_input_dict = k5k_first_author_df[["new_given","new_family","SA_list"]].to_dict("index")

In [5]:
# Full Loop - Improved:

k5k_author_out_list= list()

avail_api_keys_list = ["8707db153e4b9672fa6df25b03a5f747","2dab4694b4347fa574d159bb97484fc4","6af3d2c09eb08ec6e12f0cead9a1f5bb","3872e798bf48fa28af583b9ebef5deb6","1473c31dbcdb425f9cdaf75c673279d3"]

used_api_keys_list = list()

current_key = avail_api_keys_list.pop(0)
client = ElsClient(current_key)

print(f'Start Time = {datetime.now().strftime("%H:%M:%S")}')

for auth_id in k5k_author_input_dict:
    given_in = k5k_author_input_dict[auth_id]["new_given"]
    family_in = k5k_author_input_dict[auth_id]["new_family"]
    sa_in = k5k_author_input_dict[auth_id]["SA_list"]

    try:
        auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
        auth_srch.execute(client)
        cnt_check = True

    except Exception as e:
        err_no = re.search("\d+",str(e))[0]

        print(f"Encountered Error: {err_no}!")
        print(f'Time = {datetime.now().strftime("%H:%M:%S")}')

        with open("k5k_author_out_list_loop_save","wb") as p:
                pickle.dump(k5k_author_out_list, p)
        print("Loop save complete!")

        if "400" in str(e):
            # query error, need to skip
            print(f"Skipping author: {given_in} {family_in} - {auth_id}")
            cnt_check = False

        elif "401" in str(e):
            # VPN error, sleep try again
            sleep(60)
            print("Slept for 60 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                print("Got an error again & breaking the loop ! :(")
                break
            
        elif "429" in str(e):
            # change API key 
            while len(avail_api_keys_list) >0:
                used_api_keys_list.append(current_key)
                current_key = avail_api_keys_list.pop(0)
                client = ElsClient(current_key)
                print(f"Changed API key, now using key: {current_key}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("New key is OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "429":
                        print("New key is also finished!")
                    else:
                        print(f"Skipping author: {given_in} {family_in} - {auth_id}")
                        cnt_check = False
                        break

            if len(avail_api_keys_list) == 0:
                print("No available API keys left & breaking the loop! :(")
                break

        elif "500" in str(e):
            sleep(30)
            print("Slept for 30 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                cnt_check = False
            pass
        
        elif "10060" in str(e):
            cnt_check = False
            for i in range(10):
                sleep(30)
                print("Slept for 30 & trying again!")
                print(f"Trial number: {i+1}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("It is now OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "10060":
                        print(f"It is still {err_no}!")
                    else:
                        print(f"Another error: {err_no}")
            
            if not cnt_check:
                print(f"Couldn't fix {err_no} & breaking the loop! :(")
                break

        
        else:
            print("Unknown error & breaking the loop! :(")
            break



    if cnt_check:
        country_out = list()
        given_out = list()
        family_out = list()
        doc_count_out = list()

        for auth in auth_srch.results:
            # Country:
            try:
                country_out.append(auth['affiliation-current']['affiliation-country'])
            except:
                pass

            # Given:
            try:
                given_out.append(auth['preferred-name']['given-name'])
            except:
                pass

            # Family:
            try:
                family_out.append(auth['preferred-name']['surname'])
            except:
                pass

            # Doc Count:
            try:
                doc_count_out.append(auth['document-count'])
            except:
                pass

        author_out_dict = dict(author_id=auth_id, country=country_out, given=given_out, family=family_out, doc_count=doc_count_out)
        k5k_author_out_list.append(author_out_dict)




Start Time = 23:29:00
Encountered Error: 429!
Time = 23:29:02
Loop save complete!
Changed API key, now using key: 2dab4694b4347fa574d159bb97484fc4
New key is also finished!
Changed API key, now using key: 6af3d2c09eb08ec6e12f0cead9a1f5bb
New key is OK!
Encountered Error: 429!
Time = 23:48:58
Loop save complete!
Changed API key, now using key: 3872e798bf48fa28af583b9ebef5deb6
New key is OK!
Encountered Error: 429!
Time = 03:07:30
Loop save complete!
Changed API key, now using key: 1473c31dbcdb425f9cdaf75c673279d3
New key is OK!
No available API keys left & breaking the loop! :(


In [16]:
k5k_first_author_df

,new_given,new_family,SA_list,DOI
index,,,,
aut_0,a,alqerban,"[Dentistry, Medicine]",[10.1259/dmfr.20130157]
aut_1,a,cassioli,"[Health Professions, Medicine]",[10.1088/0031-9155/58/2/301]
aut_2,a,dietrich,"[Medicine, Nursing]",[10.1038/ijo.2016.28]
aut_3,a,drewnowski,"[Medicine, Nursing]",[10.1038/ijo.2013.179]
aut_4,a,finlayson,"[Business, Management and Accounting, Decision...",[10.1057/jors.2012.127]
...,...,...,...,...
aut_5587,zorana,jančić,"[Computer Science, Engineering, Decision Scien...",[10.1016/j.ins.2010.12.008]
aut_5588,zoë,avner,"[Social Sciences, Medicine, Health Professions]",[10.1080/13573322.2020.1782881]
aut_5589,zs.,szendro,"[Agricultural and Biological Sciences, Veterin...",[10.1016/j.livsci.2010.11.012]


In [17]:
with open("k5k_author_out_list","wb") as p:
    pickle.dump(k5k_author_out_list, p)

Only 300 records left in this loop!

In [10]:
k5k_author_input_dict = k5k_first_author_df[["new_given","new_family","SA_list"]][5202:].to_dict("index")

In [13]:
# Full Loop - Improved:

# k5k_author_out_list= list()

avail_api_keys_list = ["b7e95f5ea731eb5c9e84e7a1d499f50e", "30195f0b2192052a36bcab9ce3c4064f"]

# used_api_keys_list = list()

current_key = avail_api_keys_list.pop(0)
client = ElsClient(current_key)

print(f'Start Time = {datetime.now().strftime("%H:%M:%S")}')

for auth_id in k5k_author_input_dict:
    given_in = k5k_author_input_dict[auth_id]["new_given"]
    family_in = k5k_author_input_dict[auth_id]["new_family"]
    sa_in = k5k_author_input_dict[auth_id]["SA_list"]

    try:
        auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
        auth_srch.execute(client)
        cnt_check = True

    except Exception as e:
        err_no = re.search("\d+",str(e))[0]

        print(f"Encountered Error: {err_no}!")
        print(f'Time = {datetime.now().strftime("%H:%M:%S")}')

        with open("k5k_author_out_list_loop_save","wb") as p:
                pickle.dump(k5k_author_out_list, p)
        print("Loop save complete!")

        if "400" in str(e):
            # query error, need to skip
            print(f"Skipping author: {given_in} {family_in} - {auth_id}")
            cnt_check = False

        elif "401" in str(e):
            # VPN error, sleep try again
            sleep(60)
            print("Slept for 60 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                print("Got an error again & breaking the loop ! :(")
                break
            
        elif "429" in str(e):
            # change API key 
            while len(avail_api_keys_list) > 0:
                used_api_keys_list.append(current_key)
                current_key = avail_api_keys_list.pop(0)
                client = ElsClient(current_key)
                print(f"Changed API key, now using key: {current_key}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("New key is OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "429":
                        print("New key is also finished!")
                    else:
                        print(f"Skipping author: {given_in} {family_in} - {auth_id}")
                        cnt_check = False
                        break

            if len(avail_api_keys_list) == 0:
                print("No available API keys left & breaking the loop! :(")
                break

        elif "500" in str(e):
            sleep(30)
            print("Slept for 30 & trying again!")
            try:
                auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                auth_srch.execute(client)
                cnt_check = True
            except:
                cnt_check = False
            pass
        
        elif "10060" in str(e):
            cnt_check = False
            for i in range(10):
                sleep(30)
                print("Slept for 30 & trying again!")
                print(f"Trial number: {i+1}")
                try:
                    auth_srch = ElsSearch(f'AUTHLAST({family_in}) AND AUTHFIRST({given_in}) AND {sbj_area_query_creator(sa_in)}','author')
                    auth_srch.execute(client)
                    cnt_check = True
                    print("It is now OK!")
                    break
                except Exception as e:
                    err_no = re.search("\d+",str(e))[0]
                    if err_no == "10060":
                        print(f"It is still {err_no}!")
                    else:
                        print(f"Another error: {err_no}")
            
            if not cnt_check:
                print(f"Couldn't fix {err_no} & breaking the loop! :(")
                break

        
        else:
            print("Unknown error & breaking the loop! :(")
            break



    if cnt_check:
        country_out = list()
        given_out = list()
        family_out = list()
        doc_count_out = list()

        for auth in auth_srch.results:
            # Country:
            try:
                country_out.append(auth['affiliation-current']['affiliation-country'])
            except:
                pass

            # Given:
            try:
                given_out.append(auth['preferred-name']['given-name'])
            except:
                pass

            # Family:
            try:
                family_out.append(auth['preferred-name']['surname'])
            except:
                pass

            # Doc Count:
            try:
                doc_count_out.append(auth['document-count'])
            except:
                pass

        author_out_dict = dict(author_id=auth_id, country=country_out, given=given_out, family=family_out, doc_count=doc_count_out)
        k5k_author_out_list.append(author_out_dict)




Start Time = 08:30:28


In [15]:
pd.DataFrame(k5k_author_out_list)

,author_id,country,given,family,doc_count
0,aut_0,[Saudi Arabia],[Ali],[Alqerban],[48]
1,aut_1,[Denmark],[Andrea],[Cassioli],[17]
2,aut_2,"[United States, United States, Austria, German...","[Andrea M., Allen J., Dietrich, Arne, Alexande...","[Dietrich, Dietrich, Albert, Dietrich, Dietric...","[171, 158, 148, 131, 112, 69, 30, 22, 19, 11, ..."
3,aut_3,"[United States, United States]","[Adam, Adam]","[Drewnowski, Drewnowski]","[455, 1]"
4,aut_4,[United Kingdom],[A.],[Finlayson],[1]
...,...,...,...,...,...
5587,aut_5587,[Serbia],[Zorana],[Jančić],[9]
5588,aut_5588,[United Kingdom],[Zoë],[Avner],[7]
5589,aut_5589,[Hungary],[Zs],[Szendrő],[135]
5590,aut_5590,[China],[Zucheng],[Luo],[12]


In [18]:
with open("k5k_author_out_list_loop_save","wb") as p:
    pickle.dump(k5k_author_out_list, p)

### Part 6: Combining author_dfs

In [2]:
# Loading variables:

# Main:
with open("q1_author_df", "rb") as fp:
    q1_author_df = pickle.load(fp)

with open("q1_first_author_df", "rb") as fp:
    q1_first_author_df = pickle.load(fp)

with open("author_out_list", "rb") as fp:
    author_out_list = pickle.load(fp)


# New (20K):
with open("7d_variables/7d2_variables/new_author_df", "rb") as fp:
    new_author_df = pickle.load(fp)

with open("7d_variables/7d2_variables/new_first_author_df", "rb") as fp:
    new_first_author_df = pickle.load(fp)

with open("7d_variables/7d2_variables/new_author_out_list", "rb") as fp:
    new_author_out_list = pickle.load(fp)


# k5k (5k):
with open("7d_variables/7d2_variables/k5k_author_df", "rb") as fp:
    k5k_author_df = pickle.load(fp)

with open("7d_variables/7d2_variables/k5k_first_author_df", "rb") as fp:
    k5k_first_author_df = pickle.load(fp)

with open("7d_variables/7d2_variables/k5k_author_out_list", "rb") as fp:
    k5k_author_out_list = pickle.load(fp)

In [8]:
q1_first_author_df = q1_first_author_df.merge(pd.DataFrame(author_out_list).set_index("author_id"), how="left", left_index=True, right_index=True)

In [10]:
new_first_author_df = new_first_author_df.merge(pd.DataFrame(new_author_out_list).set_index("author_id"), how="left", left_index=True, right_index=True)

In [12]:
k5k_first_author_df = k5k_first_author_df.merge(pd.DataFrame(k5k_author_out_list).set_index("author_id"), how="left", left_index=True, right_index=True)

In [14]:
all_first_author_df = pd.concat([q1_first_author_df,new_first_author_df,k5k_first_author_df], ignore_index=True, verify_integrity=True)

In [15]:
all_first_author_df.country.isna().sum()

10

In [17]:
(all_first_author_df[all_first_author_df.country.notna()].country.map(len) == 0).sum()

6618

In [4]:
pd.DataFrame(author_out_list)

,author_id,country,given,family,doc_count
0,aut_0,"[China, China, China, Hong Kong]","[Jing, Jingheng, Jing, (Luna) Jing]","[Cai, Cai, Cai, Cai]","[23, 22, 1, 1]"
1,aut_1,"[United States, United States, United States, ...","[Qing, Qing, (Grace) Qing, Grace Qing]","[Hao, Hao, Hao, Hao]","[86, 11, 2, 1]"
2,aut_2,"[China, China, China, China, China, China, Chi...","[Yongquan, Yonghong, Yong, Dayong, Yongwu, Yon...","[Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zhou, Zho...","[286, 220, 193, 170, 164, 151, 100, 92, 74, 64..."
3,aut_3,"[Italy, Italy, France, Nigeria, Malaysia, Saud...","[Andrea, Alessandro, Ado Adamou Abba, Hafizull...","[Abba, Abbà, Ari, Ahmed, Haruna, Abba, Abba, A...","[75, 65, 44, 19, 17, 15, 8, 3]"
4,aut_4,[Italy],[Antonella],[Agodi],[234]
...,...,...,...,...,...
146526,aut_146533,[],[],[],[]
146527,aut_146534,[Slovenia],[Živa Bricman],[Rejc],[7]
146528,aut_146535,[Uzbekistan],[A. Kh],[Inoyatov],[47]
146529,aut_146536,[],[],[],[]


In [19]:
all_author_df = pd.concat([q1_author_df,new_author_df,k5k_author_df], ignore_index=True, verify_integrity=True)

In [20]:
with open("q1_author_df","wb") as p:
    pickle.dump(all_author_df, p)

with open("q1_first_author_df","wb") as p:
    pickle.dump(all_first_author_df, p)


In [21]:
with open("q1_author_df_copy","wb") as p:
    pickle.dump(all_author_df, p)

with open("q1_first_author_df_copy","wb") as p:
    pickle.dump(all_first_author_df, p)


---

This nb is finished! All necessary function are copied to 7f & all necessary variables are SAVED & DUMPED!

I'd like to thank this nb for its amazing service!

God speed nb!